<a href="https://colab.research.google.com/github/Mgalvaz/digit-recognizer/blob/main/src/train_classic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.losses import CategoricalCrossentropy
from keras import layers, models
from keras.datasets import mnist
from keras.utils import to_categorical
import tensorflow as tf
import json

In [4]:
# Import the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Modify the images so that layers can "understand" the data
train_images = train_images.reshape((-1, 28, 28, 1)).astype('float32')
test_images = test_images.reshape((-1, 28, 28, 1)).astype('float32')

# Labels one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [9]:
# Create both CNN models
model1 = models.Sequential([
    layers.Input((28, 28, 1)),
    layers.Rescaling(1./255),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='linear')
    ])
model2 = models.Sequential([
    layers.Input((28,28,1)),
    layers.Rescaling(1./255),
    layers.Conv2D(128, kernel_size=(5,5), strides=(1,1), padding='same', activation='relu'),
    layers.MaxPooling2D(strides=(2,2), padding='valid'),
    layers.Conv2D(64, kernel_size=(4,4), strides=(1,1), padding='same', activation='relu'),
    layers.MaxPooling2D(strides=(2,2), padding='valid'),
    layers.Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'),
    layers.MaxPooling2D(strides=(2,2), padding='valid'),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(10, activation='linear')
])

In [10]:
# Train the models
model1.compile(optimizer='adam', loss=CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model2.compile(optimizer='adam', loss=CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
history1 = model1.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split=0.1)
history2 = model2.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split=0.1)

# Evaluate the models
test_loss1, test_acc1 = model1.evaluate(test_images, test_labels)
test_loss2, test_acc2 = model2.evaluate(test_images, test_labels)

Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.8718 - loss: 0.4425 - val_accuracy: 0.9802 - val_loss: 0.0678
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9818 - loss: 0.0597 - val_accuracy: 0.9875 - val_loss: 0.0415
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9877 - loss: 0.0388 - val_accuracy: 0.9873 - val_loss: 0.0416
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9906 - loss: 0.0294 - val_accuracy: 0.9872 - val_loss: 0.0410
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9929 - loss: 0.0228 - val_accuracy: 0.9903 - val_loss: 0.0350
Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - accuracy: 0.7464 - loss: 0.7565 - val_accuracy: 0.9848 - val_loss: 0.0493
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9646 - loss: 0.1225 - val_accuracy: 0.9912 - val_loss: 0.0331
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9765 - loss: 0.0824 - val_accuracy: 0.9912 -

In [11]:
def save_history(history, test_loss, test_acc, filename):
    data = {
        'history': history.history,
        'test': {'test_loss': test_loss, 'test_accuracy': test_acc}
    }
    with open(filename, 'w') as f:
        json.dump(data, f)

model1.save('mnist_cnn1.keras')
model2.save('mnist_cnn2.keras')

# Save info
save_history(history1, test_loss1, test_acc1, 'history1.json')
save_history(history2, test_loss2, test_acc2, 'history2.json')